In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
tf.__version__

'2.0.0-beta1'

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [0]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.13569463789463043, Accuracy: 95.94833374023438, Test Loss: 0.0608653761446476, Test Accuracy: 97.87999725341797
Epoch 2, Loss: 0.03994172066450119, Accuracy: 98.80166625976562, Test Loss: 0.06867470592260361, Test Accuracy: 97.80999755859375
Epoch 3, Loss: 0.018533768132328987, Accuracy: 99.42666625976562, Test Loss: 0.05451153218746185, Test Accuracy: 98.38999938964844
Epoch 4, Loss: 0.013716934248805046, Accuracy: 99.55500030517578, Test Loss: 0.07139734923839569, Test Accuracy: 98.13999938964844
Epoch 5, Loss: 0.009999454952776432, Accuracy: 99.66166687011719, Test Loss: 0.06276872754096985, Test Accuracy: 98.29999542236328


In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

for i in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print(f'Dropout: {i}')
    model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5)

    model.evaluate(x_test, y_test)

Dropout: 0.1
Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3942 - accuracy: 0.8838
Epoch 2/5
60000/60000 [==============================] - 5s 81us/sample - loss: 0.2233 - accuracy: 0.9341
Epoch 3/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.1846 - accuracy: 0.9447
Epoch 4/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.1622 - accuracy: 0.9504
Epoch 5/5
10000/10000 [==============================] - 0s 44us/sample - loss: 0.0936 - accuracy: 0.9726
Dropout: 0.2
Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3899 - accuracy: 0.8846
Epoch 2/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.2240 - accuracy: 0.9346
Epoch 3/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.1866 - accuracy: 0.9437
Epoch 4/5
60000/60000 [==============================] - 5s 83us/sample - loss: 